In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import geopandas as gpd
from tools import load_ana_data
from glob import glob
from tqdm import tqdm

In [ ]:
stations = pd.read_excel("../data/external/hydroweb/stations.xlsx")
stations["Longitude"], stations["Latitude"] = stations["lon"], stations["lat"]
stations = stations.drop(["lon", "lat"], axis=1)
stations = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.Longitude, stations.Latitude))
where = (
    (stations.Latitude>-30.37)&(stations.Latitude<-29.64)&
    (stations.Longitude>-51.59)&(stations.Longitude<-50.8)
)
stations = stations[where]
stations_xr = stations[["Longitude", "Latitude", "Nome", "Codigo"]].reset_index(drop=True).to_xarray().rename(index="station")
stations.head()

In [ ]:
variables = ["water_frac", "water_frac_uncert", "classification", "illumination_time", "height","geoid"]
dlon = 20/(111.2e3*np.cos(stations.Latitude.mean()*np.pi/180))
dlat = 20/(111.2e3)

selected = {station:[] for station in stations_xr.station.values}

fnames = glob(f"../data/external/swot/*")
fnames.sort()
fnames = np.array(fnames)

dmax = 20
dsp = []
for fname in tqdm(fnames):
    dsi = xr.open_dataset(fname, group="pixel_cloud")
    
    dx = ((dsi.longitude-stations_xr.Longitude)*(111.2e3*np.cos(stations.Latitude.mean()*np.pi/180)))
    dy = ((dsi.latitude-stations_xr.Latitude)*(111.2e3))
    dist = np.sqrt(dx**2 + dy**2)
    
    if (dist.min("points")<dmax).sum()>1:
        ind = np.argwhere((dist<dmax).any("points").values).ravel()
        dist = dist.isel(station=ind)

        for i in dist.station.values:
            disti = dist.sel(station=i)
            ind = np.argwhere((disti<dmax).values).ravel()
            dspi = dsi[variables].isel(points=ind)
            dspi = dspi.assign_coords(points=np.arange(dspi.points.size))
            dspi = dspi.assign(distance=disti.isel(points=ind))
            selected[i].append(dspi)
            
for i in selected:
    dsi = xr.concat(selected[i], "obs")
    dsi = dsi.assign_coords(obs=np.arange(dsi.obs.size))
    selected[i] = dsi
selected = xr.concat([selected[i] for i in selected], "station")
selected = selected.assign_coords(time=selected.illumination_time.mean("points"))
selected = selected.assign_coords(name=stations_xr.Nome)
selected["time"] = selected["time"]-np.timedelta64(3,"h")

In [ ]:
# fname = "../data/external/hydroweb/5-87242000-TERMINAL CATSUL GUAÍBA-2024-07-09.xls"
fname = "../data/external/hydroweb/5-87450004-CAIS MAUÁ C6-2024-05-23.xlsx"
ana = load_ana_data(fname)
ana

In [ ]:
fnames = glob(f"../data/external/swot/*")
fnames.sort()
fnames = np.array(fnames)

ds = xr.open_dataset(fnames[-2], group="pixel_cloud")
ds["latitude"].attrs["long_name"] = "latitude"
ds["longitude"].attrs["long_name"] = "longitude"
ds["latitude"].attrs["units"] = "°"
ds["longitude"].attrs["units"] = "°"
ds.load()


In [ ]:
import hvplot.xarray
var = "classification"
kw = {
    "title": "",
    "x": "longitude",
    "y": "latitude",
    "geo": True,
    "rasterize": True,
    "cmap": [
        [0, 160, 0],     # land - green
        [255, 230, 90],  # land near water - dark yellow
        [100, 200, 255], # water near land - very bright blue
        [0, 0, 255],     # open water - blue
        [14, 27, 102],   # dark water - dark blue
        [192, 192, 192], # low coherence water near land - grey
        [255, 150, 255], # open low coherence water - very bright pink
    ],
    "clim": (1, 7),
    "rescale_discrete_levels": True,
    "x_sampling": 0.0002,
    "y_sampling": 0.0002,
    "aggregator": "mean",
    "colorbar": False
}

In [ ]:
import pandas as pd
import hvplot.pandas

# Your longitude and latitude
longitude = ana.longitude.values
latitude = ana.latitude.values

# Create a DataFrame for your point
df = pd.DataFrame({"Longitude": [longitude], "Latitude": [latitude]})

In [ ]:
# Generate the map
map_plot = df.hvplot.points(
    "Longitude", 
    "Latitude",
    color="red",
    geo=True,  # Enable geographic plotting
)

ds[[var]].hvplot.points(**kw)*map_plot

In [ ]:
swot = selected.where(selected.name==fname.split("-")[-4], drop=True)